In [26]:
import pandas as pd

In [27]:
import numpy as np

In [28]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [29]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import f1_score ## F1 Score 구하기
from sklearn.metrics import precision_recall_fscore_support as sk
from imblearn.over_sampling import SMOTE

#먼저 데이터 읽어오기
df=pd.read_excel('../optimal_data2/'+'Continous_2weeks_2day_1term.xlsx')
df.head()
X=df.iloc[:,[1,3,4,5,6,7]]
y=df.iloc[:,-1]

scaler = StandardScaler()
X = scaler.fit_transform(X)
X=pd.DataFrame(X)

#결과 넣을 배열
Result=[[0 for j in range(4)] for i in range(10)]

In [30]:
#한 사람당 데이터 수
Count_1=int(83*0.1)
#한 사람당 데이터 수
Count_2=83

X_test=pd.DataFrame()
X_train=pd.DataFrame()
y_test=pd.DataFrame()
y_train=pd.DataFrame()
empty=pd.DataFrame()

for i in range(10):
    X_test=empty
    X_train=empty
    y_test=empty
    y_train=empty
    for j in range(322):
        X_temp_test=X.iloc[Count_2*j+Count_1*i:Count_2*j+Count_1*(i+1)]
        X_test=pd.concat([X_test,X_temp_test])
        X_temp_train=X.iloc[Count_2*j+Count_1:Count_2*(j+1)]
        X_train=pd.concat([X_train,X_temp_train])
        
        y_temp_test=y.iloc[Count_2*j+Count_1*i:Count_2*j+Count_1*(i+1)]
        y_test=pd.concat([y_test,y_temp_test])
        y_temp_train=y.iloc[Count_2*j+Count_1:Count_2*(j+1)]
        y_train=pd.concat([y_train,y_temp_train])
     
    
    #SMOTE 적용
    smote = SMOTE(random_state=0)
    X_train,y_train = smote.fit_resample(X_train,y_train)
    X_test,y_test = smote.fit_resample(X_test,y_test)

    
    #모든 라벨 데이터 일차원으로 변환
    
    y_train=y_train.to_numpy()
    y_train=np.ravel(y_train, order='C')
    
    y_test=y_test.to_numpy()
    y_test=np.ravel(y_test, order='C')
    
    
    #모델 정의, 예측
    #여기서 문제 생기는듯
    model = SVC(kernel='rbf', C=8, gamma=0.1)
    model.fit(X_train,y_train) # rbf Kernel
    predict=model.predict(X_test)
        
    
    #Accuracy
    print("[{}]Accuracy : {}".format(i,model.score(X_test,y_test)))

    #f1score
    f1 = f1_score(y_test,predict, average='weighted')
    print("[{}]F1score : {}".format(i,f1))
    
    #precision/recall
    list=sk(y_test,predict,average='weighted')
    print("[{}]Precision : {}".format(i,list[0]))
    print("[{}]Recall : {}".format(i,list[1]))
    print("")

    #결과 배열에 넣기
    Result[i][0]=model.score(X_test,y_test)
    Result[i][1]=f1
    Result[i][2]=list[0]
    Result[i][3]=list[1]
    

[0]Accuracy : 0.5229985443959243
[0]F1score : 0.5194061587344537
[0]Precision : 0.5278812892409833
[0]Recall : 0.5229985443959243

[1]Accuracy : 0.6067763250386616
[1]F1score : 0.6073970443415795
[1]Precision : 0.6084336736320582
[1]Recall : 0.6067763250386616

[2]Accuracy : 0.6852578068264342
[2]F1score : 0.6861595737985
[2]Precision : 0.6903668681789965
[2]Recall : 0.6852578068264342

[3]Accuracy : 0.5681134507950151
[3]F1score : 0.5691385159753548
[3]Precision : 0.5741803225407697
[3]Recall : 0.5681134507950151

[4]Accuracy : 0.6062668368070324
[4]F1score : 0.6085817123972413
[4]Precision : 0.6144631609763322
[4]Recall : 0.6062668368070324

[5]Accuracy : 0.5990480190396192
[5]F1score : 0.6043667964538743
[5]Precision : 0.6168005970489003
[5]Recall : 0.5990480190396192

[6]Accuracy : 0.5947630922693267
[6]F1score : 0.598307093010659
[6]Precision : 0.6085087272668581
[6]Recall : 0.5947630922693267

[7]Accuracy : 0.5990953947368421
[7]F1score : 0.6061281793350602
[7]Precision : 0.62351

In [31]:
Result_df=pd.DataFrame(Result,columns=['Accuracy','F1-Score','Precision','Recall'])
Result_df

,Accuracy,F1-Score,Precision,Recall
0,0.522999,0.519406,0.527881,0.522999
1,0.606776,0.607397,0.608434,0.606776
2,0.685258,0.686160,0.690367,0.685258
3,0.568113,0.569139,0.574180,0.568113
4,0.606267,0.608582,0.614463,0.606267
5,0.599048,0.604367,0.616801,0.599048
6,0.594763,0.598307,0.608509,0.594763
7,0.599095,0.606128,0.623515,0.599095
8,0.663464,0.666518,0.677399,0.663464
9,0.695030,0.696647,0.707395,0.695030


In [32]:
print("Average of Accuracy {}".format(Result_df['Accuracy'].mean()))
print("Average of F1-Score {}".format(Result_df['F1-Score'].mean()))
print("Average of Precision {}".format(Result_df['Precision'].mean()))
print("Average of Recall {}".format(Result_df['Recall'].mean()))

Average of Accuracy 0.614081328809745
Average of F1-Score 0.6162650888179642
Average of Precision 0.6248944031705057
Average of Recall 0.614081328809745


In [33]:

Matrix=pd.DataFrame(Result_df['Accuracy'],columns=['Accuracy'])
Matrix['Accuracy']=Result_df['Accuracy']
A=[Result_df['Accuracy'].mean(),Result_df['F1-Score'].mean(),Result_df['Precision'].mean(),Result_df['Recall'].mean()]
A=pd.DataFrame(A,columns=['Accuracy'])
Matrix=pd.concat([Matrix,A])
Matrix=Matrix.transpose()
Matrix.to_excel('./PFMatrix.xlsx')




